In [1]:
from BERT_geoparser.data import Data
from BERT_geoparser.tokenizer import Tokenizer
from BERT_geoparser.model import BertModel

import numpy as np
import pandas as pd

from tqdm import tqdm
from sklearn.metrics import f1_score

In [3]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [142]:
# Load the dataset using the BERT_geoparser Data.py module
data_csv = r'yelp_tar_inc_tagged_bbx_org_geo.csv'
tokenizer = Tokenizer(size='base', cased=False)
data = Data(data_path=data_csv, 
            tokenizer=tokenizer,
            max_len=125)

model = BertModel(saved_model=False, data=data)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.
C:\Users\Jshin\AppData\Roaming\Python\Python39\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning

In [143]:
model.train(n_epochs=5, save_as='target_tagged_BERT_model.hdf5')

Epoch 1/5
192/192 [==============================] - 68s 251ms/step - loss: 0.1316 - accuracy: 0.7889 - val_loss: 0.0778 - val_accuracy: 0.7907
Epoch 2/5
192/192 [==============================] - 45s 234ms/step - loss: 0.0642 - accuracy: 0.8077 - val_loss: 0.0697 - val_accuracy: 0.7949
Epoch 3/5
192/192 [==============================] - 45s 235ms/step - loss: 0.0420 - accuracy: 0.8155 - val_loss: 0.0674 - val_accuracy: 0.7970
Epoch 4/5
192/192 [==============================] - 44s 228ms/step - loss: 0.0301 - accuracy: 0.8204 - val_loss: 0.0705 - val_accuracy: 0.7963
Epoch 5/5
192/192 [==============================] - 43s 225ms/step - loss: 0.0229 - accuracy: 0.8231 - val_loss: 0.0787 - val_accuracy: 0.7969


In [144]:
# Load the dataset using the BERT_geoparser Data.py module
data_csv = r'test_yelp_tar_inc_tagged_bbx_org_geo.csv'
tokenizer = Tokenizer(size='base', cased=False)
data = Data(data_path=data_csv, 
            tokenizer=tokenizer,
            max_len=125)

model = BertModel(saved_model='target_tagged_BERT_model.hdf5', data=data)

In [145]:
test_data = pd.read_json('../data/yelp_academic_dataset_review.json', lines=True, nrows=11000)[10000:]
business_df = pd.read_json('../data/yelp_academic_dataset_business.json', lines=True)
test_data = test_data.reset_index()

X_test = test_data.text
y_test = pd.read_csv('test_yelp_tar_inc_tagged_bbx_org_geo.csv')
out = model.results_dataframe(test_data.text)

100%|██████████| 1000/1000 [01:56<00:00,  8.59it/s]


In [147]:
out.head()

,sentence,token,B-inc,B-tar,I-inc,I-tar,O
0,0,[CLS],0.0,0.0,0.0,0.0,1.0
1,0,a,0.0,0.0,0.0,0.0,1.0
2,0,nice,0.0,0.0,0.0,0.0,1.0
3,0,neighborhood,0.0,0.0,0.0,0.0,1.0
4,0,feel,0.0,0.0,0.0,0.0,1.0


In [175]:
out

,sentence,token,B-inc,B-tar,I-inc,I-tar,O,true,tagged,pred,...,inc_correct,inc_TP,inc_TN,inc_FP,inc_FN,O_correct,O_TP,O_TN,O_FP,O_FN
0,0,[CLS],0.0,0.0,0.0,0.0,1.0,O,False,B-tar,...,False,False,True,False,False,False,False,False,False,True
1,0,a,0.0,0.0,0.0,0.0,1.0,O,False,B-tar,...,False,False,True,False,False,False,False,False,False,True
2,0,nice,0.0,0.0,0.0,0.0,1.0,O,False,B-tar,...,False,False,True,False,False,False,False,False,False,True
3,0,neighborhood,0.0,0.0,0.0,0.0,1.0,O,False,B-tar,...,False,False,True,False,False,False,False,False,False,True
4,0,feel,0.0,0.0,0.0,0.0,1.0,O,False,B-tar,...,False,False,True,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84664,999,the,0.0,0.0,0.0,0.0,1.0,O,False,B-tar,...,False,False,True,False,False,False,False,False,False,True
84665,999,other,0.0,0.0,0.0,0.0,1.0,O,False,B-tar,...,False,False,True,False,False,False,False,False,False,True
84666,999,hand,0.0,0.0,0.0,0.0,1.0,O,False,B-tar,...,False,False,True,False,False,False,False,False,False,True
84667,999,",",0.0,0.0,0.0,0.0,1.0,O,False,B-tar,...,False,False,True,False,False,False,False,False,False,True


In [190]:
#out_tagged = [True if x.sentence in y_test['Sentence #'].unique() else False for i,x in out.iterrows()]
out['tagged'] = out_tagged
# add predicted and true tags
tags = ['B-tar', 'B-inc', 'I-inc', 'I-tar', 'O']
out['true'] = 'O'
out.loc[out.tagged==True, 'true'] = y_test['Tag']
out['pred'] = out[tags].idxmax(axis=1)
# seems to add an NA row at the bottom, not sure why...
out = out.dropna()

In [201]:
out['correct'] = [True if x.true == x.pred else False for i,x in out.iterrows() ]
#target
for cat in ['tar', 'inc', 'O']:
    out[f'{cat}_correct'] = [True if (x.true == x.pred) and (cat in x.true) else False for i,x in out.iterrows()] 
    out[f'{cat}_TP'] = [True if (x.true == x.pred) and (cat in x.true) else False for i,x in out.iterrows()]
    out[f'{cat}_TN'] = [True if (x.true != x.pred) and (cat not in x.true) else False for i,x in out.iterrows()] 
    out[f'{cat}_FP'] = [True if (x.true == x.pred) and (cat not in x.true) else False for i,x in out.iterrows()] 
    out[f'{cat}_FN'] = [True if (x.true != x.pred) and (cat in x.true) else False for i,x in out.iterrows()] 

In [202]:
accuracy = sum(out.correct)/len(out)
results = {}
for cat in ['tar', 'inc', 'O']:
    results[f'{cat}_accuracy'] = sum(out[f'{cat}_correct'])/len(out[out.true.str.contains(cat)])
    results[f'{cat}_recall'] = sum(out[f'{cat}_TP'])/(sum(out[f'{cat}_TP']) + sum(out[f'{cat}_FN']))
    results[f'{cat}_precision'] = sum(out[f'{cat}_TP'])/(sum(out[f'{cat}_TP']) + sum(out[f'{cat}_FP']))

In [203]:
results

{'tar_accuracy': 0.020491803278688523,
 'tar_recall': 0.020491803278688523,
 'tar_precision': 6.117929203323259e-05,
 'inc_accuracy': 0.008914100486223663,
 'inc_recall': 0.008914100486223663,
 'inc_precision': 0.0001345944424731117,
 'O_accuracy': 0.9822096140207475,
 'O_recall': 0.9822096140207475,
 'O_precision': 0.9998042262654937}